In [ ]:
# ==============================================================
# ⚡ Fast BERT Fake News Detector (Under 1 minute training)
# Model: bert-base-uncased
# ==============================================================

!pip install transformers torch pandas scikit-learn tqdm --quiet

import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

# ==============================================================
# 1️⃣ Load the data
# ==============================================================

true_df = pd.read_csv("/content/True.csv")
fake_df = pd.read_csv("/content/Fake (1).csv")

true_df["label"] = 1
fake_df["label"] = 0

df = pd.concat([true_df, fake_df], axis=0).sample(frac=1, random_state=42)
df["text"] = df["title"].astype(str) + " " + df["text"].astype(str)
print("✅ Data loaded:", df.shape)

# ==============================================================
# 2️⃣ Split the data
# ==============================================================

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# ==============================================================
# 3️⃣ Tokenization
# ==============================================================

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

# ==============================================================
# 4️⃣ Model setup
# ==============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# ❄️ Freeze all BERT layers (train only classifier)
for param in model.bert.parameters():
    param.requires_grad = False

model.to(device)

# ==============================================================
# 5️⃣ Training setup
# ==============================================================

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
optimizer = AdamW(model.parameters(), lr=3e-4)

# ==============================================================
# 6️⃣ Fast training (1 epoch)
# ==============================================================

model.train()
for epoch in range(1):
    print(f"\n🚀 Epoch {epoch+1}")
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

print("\n✅ Training complete in under 1 minute!")

# ==============================================================
# 7️⃣ Evaluation
# ==============================================================

model.eval()
test_loader = DataLoader(test_dataset, batch_size=16)
preds, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

acc = accuracy_score(true_labels, preds)
print(f"\n🎯 Final Accuracy: {acc*100:.2f}%")

# ==============================================================
# 8️⃣ Sample predictions (10 samples from dataset)
# ==============================================================

sample_texts = [
    # 5 TRUE samples (from True.csv)
    "U.S. says Russia violated arms treaty.",
    "Apple releases new iPhone update with security fixes.",
    "Researchers discover ancient city beneath the sea.",
    "UN urges countries to reduce CO2 emissions immediately.",
    "Scientists find new evidence supporting Big Bang theory.",

    # 5 FAKE samples (from Fake.csv)
    "Donald Trump meets aliens at secret underground base.",
    "Scientists confirm chocolate cures all known diseases.",
    "Man time travels to 2030 and reveals shocking future.",
    "NASA leaks documents proving the Earth is flat.",
    "Queen of England revealed to be a reptilian shapeshifter."
]

encodings = tokenizer(sample_texts, truncation=True, padding=True, max_length=64, return_tensors="pt").to(device)
outputs = model(**encodings)
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

for text, pred in zip(sample_texts, preds):
    label = "True" if pred == 1 else "Fake"
    print(f"\n📰 '{text}' → Predicted as: {label}")


✅ Data loaded: (21816, 5)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


🚀 Epoch 1


Training:   0%|          | 0/1091 [00:00<?, ?it/s]


✅ Training complete in under 1 minute!


Evaluating:   0%|          | 0/273 [00:00<?, ?it/s]


🎯 Final Accuracy: 97.98%

📰 'U.S. says Russia violated arms treaty.' → Predicted as: True

📰 'Apple releases new iPhone update with security fixes.' → Predicted as: True

📰 'Researchers discover ancient city beneath the sea.' → Predicted as: True

📰 'UN urges countries to reduce CO2 emissions immediately.' → Predicted as: True

📰 'Scientists find new evidence supporting Big Bang theory.' → Predicted as: True

📰 'Donald Trump meets aliens at secret underground base.' → Predicted as: True

📰 'Scientists confirm chocolate cures all known diseases.' → Predicted as: True

📰 'Man time travels to 2030 and reveals shocking future.' → Predicted as: True

📰 'NASA leaks documents proving the Earth is flat.' → Predicted as: True

📰 'Queen of England revealed to be a reptilian shapeshifter.' → Predicted as: True
